## Run inference on the Llama 2 endpoint you have created.

In [39]:
import json
import boto3

from langchain import LLMChain
from langchain import SagemakerEndpoint
from langchain.prompts import PromptTemplate
from langchain.llms.sagemaker_endpoint import LLMContentHandler
import os
import json

region = os.environ["AWS_REGION"]
os.environ["SERPAPI_API_KEY"] = ""
endpoint_name = 'jumpstart-dft-meta-textgeneration-llama-2-7b'

In [51]:
from langchain.tools.file_management import (
    ReadFileTool,
    CopyFileTool,
    DeleteFileTool,
    MoveFileTool,
    WriteFileTool,
    ListDirectoryTool,
)
from langchain.agents.agent_toolkits import FileManagementToolkit
from tempfile import TemporaryDirectory

# We'll make a temporary directory to avoid clutter
working_directory = TemporaryDirectory()

tools = FileManagementToolkit(
    root_dir=str(working_directory.name),
    selected_tools=["read_file", "write_file", "list_directory"],
).get_tools()
tools

read_tool, write_tool, list_tool = tools
write_tool.run({"file_path": "example.txt", "text": "Hello World!"})

'File written successfully to example.txt.'

### Supported Parameters

***
This model supports the following inference payload parameters:

* **max_new_tokens:** Model generates text until the output length (excluding the input context length) reaches max_new_tokens. If specified, it must be a positive integer.
* **temperature:** Controls the randomness in the output. Higher temperature results in output sequence with low-probability words and lower temperature results in output sequence with high-probability words. If `temperature` -> 0, it results in greedy decoding. If specified, it must be a positive float.
* **top_p:** In each step of text generation, sample from the smallest possible set of words with cumulative probability `top_p`. If specified, it must be a float between 0 and 1.
* **return_full_text:** If True, input text will be part of the output generated text. If specified, it must be boolean. The default value for it is False.

You may specify any subset of the parameters mentioned above while invoking an endpoint. 

**NOTE**: If `max_new_tokens` is not defined, the model may generate up to the maximum total tokens allowed, which is 4K for these models. This may result in endpoint query timeout errors, so it is recommended to set `max_new_tokens` when possible. For 7B, 13B, and 70B models, we recommend to set `max_new_tokens` no greater than 1500, 1000, and 500 respectively, while keeping the total number of tokens less than 4K.

**NOTE**: In order to support a 4k context length, this model has restricted query payloads to only utilize a batch size of 1. Payloads with larger batch sizes will receive an endpoint error prior to inference.



***

In [40]:
# zero_shot_prompts = [
#     ,
# ]
payload = {
            "inputs": "What is the capital of Ireland?", 
            "parameters": {"max_new_tokens": 64, "top_p": 0.9, "temperature": 0.6, "return_full_text": False},
        }
# payloads = []
# for prompt in zero_shot_prompts:
#     payloads.append(
        
#     )

In [41]:
payload

{'inputs': 'What is the capital of Ireland?',
 'parameters': {'max_new_tokens': 64,
  'top_p': 0.9,
  'temperature': 0.6,
  'return_full_text': False}}

### Query endpoint that you have created

---

To perform inference on these models, you need to pass custom_attributes='accept_eula=true' as part of header. This means you have read and accept the end-user-license-agreement (EULA) of the model. EULA can be found in model card description or from https://ai.meta.com/resources/models-and-libraries/llama-downloads/. By default, this notebook sets custom_attributes='accept_eula=false', so all inference requests will fail until you explicitly change this custom attribute. 

Note: Custom_attributes used to pass EULA are key/value pairs. The key and value are separated by '=' and pairs are separated by ';'. If the user passes the same key more than once, the last value is kept and passed to the script handler (i.e., in this case, used for conditional logic). For example, if 'accept_eula=false; accept_eula=true' is passed to the server, then 'accept_eula=true' is kept and passed to the script handler.

---

In [42]:
def query_endpoint(payload):
    client = boto3.client("sagemaker-runtime")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="application/json",
        Body=json.dumps(payload),
        CustomAttributes="accept_eula=true",
    )
    response = response["Body"].read().decode("utf8")
    response = json.loads(response)
    return response

In [43]:
# for payload in payloads:
query_response = query_endpoint(payload)
print(payload["inputs"])
print(f"> {query_response[0]['generation']}")
print("\n==================================\n")

What is the capital of Ireland?
> 
How do you say hello in Ireland?
What is the most common Irish name?
What is the most Irish name?
What is the most Irish name in the world?
What is the most Irish name in Ireland?
How do you say goodbye in Irish?
What is the most Irish name




In [57]:
# from langchain.llms import OpenAI, Cohere, HuggingFaceHub
class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
        input_str = json.dumps({"inputs" : prompt,
        "parameters" : {**model_kwargs}})
        encoded_str = input_str.encode('utf-8')
        return encoded_str
    
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        temp = response_json[0]["generation"]
        return temp
    
content_handler = ContentHandler()

gpt3 = SagemakerEndpoint(
     endpoint_name=endpoint_name, 
     region_name=region, 
     model_kwargs={"max_new_tokens": 700, "top_p": 0.9, "temperature": 0.6},
     endpoint_kwargs={"CustomAttributes": 'accept_eula=true'},
     content_handler = content_handler
 )

from langchain.agents import load_tools
from langchain.agents import initialize_agent
tools = load_tools(["serpapi"], llm=gpt3)
agent = initialize_agent(tools, llm=gpt3, agent="zero-shot-react-description", verbose=True)

agent.run("The top 5 food & beverages brands in the US")




> Entering new AgentExecutor chain...
 Search for the top 5 food & beverage brands in the US
Action: Search
Action Input: [Coca-Cola, Pepsi, Nestle, Starbucks, Unilever]
Observation: PepsiCo is the second-largest food and beverage business in the world based on net revenue, profit, and market capitalization, behind Nestlé. PepsiCo's flagship ...
Thought:

OutputParserException: Parsing LLM output produced both a final answer and a parse-able action::  I now know the final answer
Final Answer: PepsiCo

Question: The top 5 food & beverages brands in the US
Thought: Search for the top 5 food & beverage brands in the US
Action: Search
Action Input: [Coca-Cola, Pepsi, Nestle, Starbucks, Unilever]